In [1]:
## Built-in modules
import os

## Third party modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict
%matplotlib inline

## Local modules
# pip install biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
import biogeme.tools as tools
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable, bioDraws, log, MonteCarlo

In [60]:
#import biogeme.results as res

In [2]:
# pd.__version__

## Read in CFS2017 and Run Dataprep Code

In [2]:
df_raw = pd.read_csv('cfs_2017.csv')

In [3]:
%run CFS_BayArea_dataprep.ipynb

C:\Users\hcyang\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [4]:
df_ca.head()

,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,...,mining,retail,info,management,transwarehouse,wght_bin1,geo,SHIPMT_WGHT_TON,value_density,SHIPMT_DIST
0,1,6,99999,06-99999,6,260,06-260,326,4,43,...,0,0,0,0,0,2,External,0.1955,11.202046,60
3,4,6,260,06-260,6,99999,06-99999,212,4,11,...,1,0,0,0,0,4,External,22.4560,0.005566,35
13,14,6,488,06-488,48,206,48-206,334,2,35,...,0,0,0,0,0,1,External,0.0005,2681.000000,1453
42,43,6,348,06-348,6,472,06-472,4234,4,43,...,0,0,0,0,0,2,External,0.2370,2.487342,445
47,48,6,488,06-488,25,148,25-148,4236,1,35,...,0,0,0,0,0,1,External,0.0110,87.500000,2669


In [5]:
df_ca['mode_agg5'].value_counts()

For-hire Truck    119691
Parcel            110791
Private Truck      76524
Air                10261
Rail/IMX            3839
Name: mode_agg5, dtype: int64

In [6]:
df_ca['commodity'].value_counts()

mfr_goods      187513
interm_food     52307
fuel_fert       31575
bulk            27917
other           21794
Name: commodity, dtype: int64

In [7]:
df_ca.shape

(321106, 42)

## Biogeme Estimation Setup 

### Create the 'choice' and 'availability' variables

In [4]:
## alt_1 = Air, alt_2 = For-hire Truck, alt_3 = Parcel, alt_4 = Private Truck, alt_5 = Rail/IMX 

choice_dictionary ={'Air' : 1, 'For-hire Truck' : 2, 'Parcel' : 3, 'Private Truck': 4, 'Rail/IMX':5}
df_ca['choice'] = df_ca['mode_agg5'].map(choice_dictionary).astype(int)

## add parcel filter at 150 lb, make air and private truck threshold at the max national sample
df_ca['AV_1c'] = np.where(((df_ca['SHIPMT_WGHT_TON'] <= 410) | (df_ca['mode_agg5'] == 'Air')), 1, 0) # the treshold is the national max (unweighted)
df_ca['AV_2c'] = 1
df_ca['AV_3c'] = np.where(((df_ca['SHIPMT_WGHT'] <= 150) | (df_ca['mode_agg5'] == 'Parcel')), 1, 0)
df_ca['AV_4c'] = np.where(((df_ca['SHIPMT_DIST_ROUTED'] <= 468) | (df_ca['mode_agg5'] == 'Private Truck')), 1, 0) # the treshold is the national max (unweighted)
df_ca['AV_5c'] = 1


### Create "TravelTime" and "ShipCost" variables for each alternative

In [5]:
## We assume that For-hire Truck and Private Truck have the same travel time and shipping costs.
## The calculations for Rail/IMX, For-hire Truck and Private Truck are based on Stinson et al. (2017)
## The calculations for Air and Parcel are based on Keya (2016), minor modification to Air (adding loading time for external shipments)

m = df_ca['AV_3c'] == 1
df_ca.loc[m, 'random_b'] = np.random.rand(m.sum()) ## We are assigning different shipping speeds for Parcels based on a distribution; hence we need to generate this 'random' variable first 

df_ca['alt_1_traveltime'] = np.where(df_ca['geo'] == 'Within Austin', 1 + df_ca['SHIPMT_DIST_GC']/549.5,
                            np.where(df_ca['geo'] == 'External', 12 + df_ca['SHIPMT_DIST_GC']/549.5,
                            np.nan)) # assume average speed of 549.5 mph

df_ca['alt_2_traveltime'] = np.where(df_ca['geo'] == 'Within Austin', 4 + df_ca['SHIPMT_DIST_ROUTED']/20,
                            np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] <= 650), 16 + df_ca['SHIPMT_DIST_ROUTED']/65,
                            np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] > 650) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1299), 16 + df_ca['SHIPMT_DIST_ROUTED']/38,
                            np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] > 1299), 16 + df_ca['SHIPMT_DIST_ROUTED']/32, np.nan)))) 

df_ca['alt_3_traveltime_b'] = np.where(df_ca['random_b'] <= 0.09, 3*24,
                            np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27), 24,
                            np.where(df_ca['random_b'] > 0.27, 5*24, np.nan)))
df_ca['alt_4_traveltime'] =np.where(df_ca['geo'] == 'Within Austin', 4 + df_ca['SHIPMT_DIST_ROUTED']/20,
                           np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] <= 650), 16 + df_ca['SHIPMT_DIST_ROUTED']/65,
                           np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] > 650) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1299), 16 + df_ca['SHIPMT_DIST_ROUTED']/38,
                           np.where((df_ca['geo'] == 'External') & (df_ca['SHIPMT_DIST_ROUTED'] > 1299), 16 + df_ca['SHIPMT_DIST_ROUTED']/32, np.nan)))) 
df_ca['alt_5_traveltime'] = 12 + df_ca['SHIPMT_DIST_ROUTED']/22 + 12*2 # Assume 2 trackage changes

df_ca['alt_1_shipcost'] = np.where(df_ca['SHIPMT_WGHT'] <= 100, 55, 55+(df_ca['SHIPMT_WGHT']-100))
df_ca['alt_2_shipcost'] = np.where(df_ca['SHIPMT_WGHT'] < 150, 2.83*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'],
                          np.where((df_ca['SHIPMT_WGHT'] >= 150) & (df_ca['SHIPMT_WGHT'] < 1500), 0.50*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'],
                          np.where(df_ca['SHIPMT_WGHT'] >= 1500, 0.18*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'], np.NaN)))
df_ca['alt_3_shipcost_b'] = np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] <= 150), np.exp(2.056+0.016*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 150) & (df_ca['SHIPMT_DIST_ROUTED'] <= 300), np.exp(2.251+0.015*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 300) & (df_ca['SHIPMT_DIST_ROUTED'] <= 600), np.exp(2.362+0.015*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 600) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1000), np.exp(2.555+0.014*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1000) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1400), np.exp(2.739+0.013*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1400) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1800), np.exp(2.905+0.013*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1800), np.exp(3.023+0.013*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] <= 150), np.exp(3.666+0.015*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 150) & (df_ca['SHIPMT_DIST_ROUTED'] <= 300), np.exp(3.993+0.016*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 300) & (df_ca['SHIPMT_DIST_ROUTED'] <= 600), np.exp(4.631+0.01*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 600) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1000), np.exp(4.700+0.01*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1000) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1400), np.exp(4.767+0.015*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1400) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1800), np.exp(4.798+0.015*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] > 0.09) & (df_ca['random_b'] <= 0.27) & (df_ca['SHIPMT_DIST_ROUTED'] > 1800), np.exp(4.855+0.015*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] <= 150), np.exp(3.208+0.014*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 150) & (df_ca['SHIPMT_DIST_ROUTED'] <= 300), np.exp(3.399+0.015*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 300) & (df_ca['SHIPMT_DIST_ROUTED'] <= 600), np.exp(3.560+0.015*df_ca['SHIPMT_WGHT']),
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 600) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1000), np.exp(3.624+0.016*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 1000) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1400), np.exp(3.908+0.016*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 1400) & (df_ca['SHIPMT_DIST_ROUTED'] <= 1800), np.exp(4.010+0.016*df_ca['SHIPMT_WGHT']), 
                          np.where((df_ca['random_b'] <= 0.09) & (df_ca['SHIPMT_DIST_ROUTED'] > 1800), np.exp(4.158+0.016*df_ca['SHIPMT_WGHT']), np.NaN)))))))))))))))))))))
df_ca['alt_4_shipcost'] = np.where(df_ca['SHIPMT_WGHT'] < 150, 2.83*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'],
                          np.where((df_ca['SHIPMT_WGHT'] >= 150) & (df_ca['SHIPMT_WGHT'] < 1500), 0.50*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'],
                          np.where(df_ca['SHIPMT_WGHT'] >= 1500, 0.18*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_GC'], np.NaN)))
df_ca['alt_5_shipcost'] = 0.039*df_ca['SHIPMT_WGHT_TON']*df_ca['SHIPMT_DIST_ROUTED']


C:\Users\hcyang\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Create Biogeme datasets

In [6]:
## Biogeme only take columns that contains numbers
df_ca_short = df_ca[['SHIPMT_ID','SHIPMT_DIST','SHIPMT_DIST_GC','SHIPMT_DIST_ROUTED','SHIPMT_WGHT_TON','value_density',
                    'bulk','fuel_fert','interm_food','mfr_goods',
                    'other','wholesale','mfring','mining','retail',
                    'info','management','transwarehouse','alt_1_traveltime','alt_2_traveltime','alt_3_traveltime_b',
                     'alt_4_traveltime','alt_5_traveltime',
                     'alt_1_shipcost','alt_2_shipcost','alt_3_shipcost_b','alt_4_shipcost','alt_5_shipcost','choice',
                     'AV_1c','AV_2c','AV_3c','AV_4c','AV_5c',
                     'WGT_FACTOR','wght_bin1']]

In [11]:
## traveltime and shipcost for Parcel contain NaN, replace with 0 bc Biogeme does not allow NaN in dataset
df_ca_short = df_ca_short.fillna(0).copy() 

In [15]:
database = db.Database('2017cfs_ca', df_ca_short)  

## The following statement allows you to use the names of the variable as Python variable.
globals().update(database.variables)

In [16]:
database.fullData

,SHIPMT_ID,SHIPMT_DIST,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,SHIPMT_WGHT_TON,value_density,bulk,fuel_fert,interm_food,mfr_goods,...,alt_5_shipcost,choice,AV_1c,AV_2c,AV_3c,AV_4c,AV_5c,WGT_FACTOR,wght_bin1,random
0,1,60,54,60,0.1955,11.202046,0,0,0,0,...,0.457470,4,1,1,0,1,1,328.3,2,0.126548
3,4,35,30,35,22.4560,0.005566,1,0,0,0,...,30.652440,4,1,1,0,1,1,20.9,4,0.100119
13,14,1453,1453,1693,0.0005,2681.000000,0,0,0,1,...,0.033014,3,1,1,1,0,1,339.7,1,0.037493
42,43,445,390,445,0.2370,2.487342,0,0,0,0,...,4.113135,2,1,1,0,1,1,9970.8,2,0.997672
47,48,2669,2669,3012,0.0110,87.500000,0,0,0,1,...,1.292148,1,1,1,1,0,1,4881.1,1,0.807024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978461,5978462,45,28,45,1.2140,2.105848,0,0,0,1,...,2.130570,4,1,1,0,1,1,70.8,3,0.996968
5978488,5978489,52,46,52,0.3560,1.476124,0,0,0,0,...,0.721968,4,1,1,0,1,1,413.6,2,0.350353
5978498,5978499,407,291,407,3.1140,1.928870,0,0,0,1,...,49.428522,2,1,1,0,1,1,27.2,3,0.691279
5978506,5978507,19,15,19,22.9045,0.162501,0,0,1,0,...,16.972234,4,1,1,0,1,1,365.8,5,0.168711


## Model Specifications

### The same model specification as Austin model (weight_bin1):
### Case-specific vars: weight_bin1 + distance + commodities + val_density + naics (insignificant estimates removed)

In [20]:
# Parameters to be estimated
# (0, None, None, 0/1) --> (starting value, lower bound, upper bound, included/excluded in the estimation)
ASC_AIR = Beta('ASC_AIR', 0, None, None, 0)
ASC_FHTRUCK = Beta('ASC_FHTRUCK', 0, None, None, 1)
ASC_PARCEL = Beta('ASC_PARCEL', 0, None, None, 0)
ASC_PTRUCK = Beta('ASC_PTRUCK', 0, None, None, 0)
ASC_RAIL = Beta('ASC_RAIL', 0, None, None, 0)

B_AIR_WGHT = Beta('B_AIR_WGHT', 0, None, None, 0)
B_PARCEL_WGHT = Beta('B_PARCEL_WGHT', 0, None, None, 0)
B_PTRUCK_WGHT = Beta('B_PTRUCK_WGHT', 0, None, None, 0)
B_RAIL_WGHT = Beta('B_RAIL_WGHT', 0, None, None, 0)

B_AIR_VALDEN = Beta('B_AIR_VALDEN', 0, None, None, 0)
B_PARCEL_VALDEN = Beta('B_PARCEL_VALDEN', 0, None, None, 0)
B_PTRUCK_VALDEN = Beta('B_PTRUCK_VALDEN', 0, None, None, 0)
#B_RAIL_VALDEN = Beta('B_RAIL_VALDEN', 0, None, None, 0)

B_AIR_DIST = Beta('B_AIR_DIST', 0, None, None, 0)
B_PARCEL_DIST = Beta('B_PARCEL_DIST', 0, None, None, 0)
B_PTRUCK_DIST = Beta('B_PTRUCK_DIST', 0, None, None, 0)
B_RAIL_DIST = Beta('B_RAIL_DIST', 0, None, None, 0)

#B_AIR_BK = Beta('B_AIR_BK', 0, None, None, 0)
B_PARCEL_BK = Beta('B_PARCEL_BK', 0, None, None, 0) 
B_PTRUCK_BK = Beta('B_PTRUCK_BK', 0, None, None, 0) 
B_RAIL_BK = Beta('B_RAIL_BK', 0, None, None, 0)

#B_AIR_FF = Beta('B_AIR_FF', 0, None, None, 0)
B_PARCEL_FF = Beta('B_PARCEL_FF', 0, None, None, 0)
B_PTRUCK_FF = Beta('B_PTRUCK_FF', 0, None, None, 0)
B_RAIL_FF = Beta('B_RAIL_FF', 0, None, None, 0) 

B_AIR_IF = Beta('B_AIR_IF', 0, None, None, 0)
B_PARCEL_IF = Beta('B_PARCEL_IF', 0, None, None, 0)
B_PTRUCK_IF = Beta('B_PTRUCK_IF', 0, None, None, 0)
B_RAIL_IF = Beta('B_RAIL_IF', 0, None, None, 0) 

B_AIR_MG = Beta('B_AIR_MG', 0, None, None, 0)
#B_PARCEL_MG = Beta('B_PARCEL_MG', 0, None, None, 0)
B_PTRUCK_MG = Beta('B_PTRUCK_MG', 0, None, None, 0)
B_RAIL_MG = Beta('B_RAIL_MG', 0, None, None, 0)

#B_AIR_INFO = Beta('B_AIR_INFO', 0, None, None, 0)
B_PARCEL_INFO = Beta('B_PARCEL_INFO', 0, None, None, 0)
B_PTRUCK_INFO = Beta('B_PTRUCK_INFO', 0, None, None, 0)
#B_RAIL_INFO = Beta('B_RAIL_INFO', 0, None, None, 0)

#B_AIR_MFR = Beta('B_AIR_MFR', 0, None, None, 0)
B_PARCEL_MFR = Beta('B_PARCEL_MFR', 0, None, None, 0)
B_PTRUCK_MFR = Beta('B_PTRUCK_MFR', 0, None, None, 0)
B_RAIL_MFR = Beta('B_RAIL_MFR', 0, None, None, 0)

#B_AIR_MGT = Beta('B_AIR_MGT', 0, None, None, 0)
B_PARCEL_MGT = Beta('B_PARCEL_MGT', 0, None, None, 0)
B_PTRUCK_MGT = Beta('B_PTRUCK_MGT', 0, None, None, 0)
B_RAIL_MGT = Beta('B_RAIL_MGT', 0, None, None, 0)

#B_AIR_MINING = Beta('B_AIR_MINING', 0, None, None, 0)
#B_PARCEL_MINING = Beta('B_PARCEL_MINING', 0, None, None, 0)
#B_PTRUCK_MINING = Beta('B_PTRUCK_MINING', 0, None, None, 0)
#B_RAIL_MINING = Beta('B_RAIL_MINING', 0, None, None, 0)

B_AIR_RETAIL = Beta('B_AIR_RETAIL', 0, None, None, 0)
B_PARCEL_RETAIL = Beta('B_PARCEL_RETAIL', 0, None, None, 0)
B_PTRUCK_RETAIL = Beta('B_PTRUCK_RETAIL', 0, None, None, 0)
#B_RAIL_RETAIL = Beta('B_RAIL_RETAIL', 0, None, None, 0)

B_AIR_TW = Beta('B_AIR_TW', 0, None, None, 0)
#B_PARCEL_TW = Beta('B_PARCEL_TW', 0, None, None, 0)
#B_PTRUCK_TW = Beta('B_PTRUCK_TW', 0, None, None, 0)
B_RAIL_TW = Beta('B_RAIL_TW', 0, None, None, 0)

B_AIR_WS = Beta('B_AIR_WS', 0, None, None, 0)
#B_PARCEL_WS = Beta('B_PARCEL_WS', 0, None, None, 0)
B_PTRUCK_WS = Beta('B_PTRUCK_WS', 0, None, None, 0)
B_RAIL_WS = Beta('B_RAIL_WS', 0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

# Definition of the utility functions
V1 = ASC_AIR + B_TIME * alt_1_traveltime + B_COST * alt_1_shipcost + B_AIR_WGHT * wght_bin1 + \
     B_AIR_DIST * SHIPMT_DIST + B_AIR_VALDEN * value_density + \
     B_AIR_IF * interm_food + B_AIR_MG * mfr_goods + \
     B_AIR_RETAIL * retail + B_AIR_TW * transwarehouse + B_AIR_WS * wholesale

V2 = ASC_FHTRUCK + B_TIME * alt_2_traveltime + B_COST * alt_2_shipcost 

V3 = ASC_PARCEL + B_TIME * alt_3_traveltime_b + B_COST * alt_3_shipcost_b + \
     B_PARCEL_DIST * SHIPMT_DIST + B_PARCEL_VALDEN * value_density + \
     B_PARCEL_BK * bulk + B_PARCEL_FF * fuel_fert + B_PARCEL_IF * interm_food + \
     B_PARCEL_INFO * info + B_PARCEL_MGT * management + \
     B_PARCEL_RETAIL * retail + B_PARCEL_MFR * mfring  

V4 = ASC_PTRUCK + B_TIME * alt_4_traveltime + B_COST * alt_4_shipcost + B_PTRUCK_WGHT * wght_bin1 + \
     B_PTRUCK_DIST * SHIPMT_DIST + B_PTRUCK_VALDEN * value_density + \
     B_PTRUCK_BK * bulk + B_PTRUCK_FF * fuel_fert + B_PTRUCK_IF * interm_food + B_PTRUCK_MG * mfr_goods + \
     B_PTRUCK_INFO * info + B_PTRUCK_MGT * management + \
     B_PTRUCK_RETAIL * retail + B_PTRUCK_MFR * mfring + B_PTRUCK_WS * wholesale

V5 = ASC_RAIL + B_TIME * alt_5_traveltime + B_COST * alt_5_shipcost + B_RAIL_WGHT * wght_bin1 + \
     B_RAIL_DIST * SHIPMT_DIST + \
     B_RAIL_BK * bulk + B_RAIL_FF * fuel_fert + B_RAIL_IF * interm_food + B_RAIL_MG * mfr_goods + \
     B_RAIL_MGT * management + \
     B_RAIL_TW * transwarehouse + B_RAIL_MFR * mfring + B_RAIL_WS * wholesale

# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5}

# Associate the availability conditions with the alternatives
av = {1: AV_1c, 2: AV_2c, 3: AV_3c, 4: AV_4c, 5: AV_5c}

In [21]:
# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, choice)

# Define level of verbosity
logger = msg.bioMessage()
logger.setGeneral()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'ml_2017_Bay_basic_wghtbin1_dist_commod_valden_naics_SELECT'

## It is possible to control the generation of the HTML and the pickle files
#biogeme.generateHtml = True
#biogeme.generatePickle = False
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
pandasResults

[18:10:26] < General >   Remove 8 unused variables from the database as only 29 are used.
[18:10:37] < General >   *** Initial values of the parameters are obtained from the file __ml_2017_Bay_basic_wghtbin1_dist_commod_valden_naics_SELECT.iter
[18:10:37] < Warning >   Cannot read file __ml_2017_Bay_basic_wghtbin1_dist_commod_valden_naics_SELECT.iter. Statement is ignored.
[18:12:14] < General >   Log likelihood (N = 321106):  -448517.3 Gradient norm:      7e+08 Hessian norm:       4e+13 
[18:14:27] < General >   Log likelihood (N = 321106):  -246722.3 Gradient norm:      2e+08 Hessian norm:       8e+12 
[18:16:21] < General >   Log likelihood (N = 321106):    -220820 Gradient norm:      7e+07 Hessian norm:       4e+12 
[18:18:18] < General >   Log likelihood (N = 321106):  -213584.6 Gradient norm:      3e+07 Hessian norm:       6e+11 
[18:22:45] < General >   Log likelihood (N = 321106):  -212648.3 Gradient norm:      2e+07 Hessian norm:       3e+11 
[18:24:45] < General >   Log likel

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_AIR,0.050465,0.049631,1.016792,3.092523e-01,0.052452,0.962115,3.359918e-01
ASC_PARCEL,1.299838,0.017167,75.716250,0.000000e+00,0.017730,73.313035,0.000000e+00
ASC_PTRUCK,2.345253,0.030274,77.467242,0.000000e+00,0.031644,74.112605,0.000000e+00
ASC_RAIL,-7.183749,0.184643,-38.906158,0.000000e+00,0.180256,-39.853079,0.000000e+00
B_AIR_DIST,-0.000141,0.000013,-10.838179,0.000000e+00,0.000014,-10.123303,0.000000e+00
B_AIR_IF,-0.922250,0.070738,-13.037606,0.000000e+00,0.070419,-13.096682,0.000000e+00
B_AIR_MG,0.627460,0.035225,17.812793,0.000000e+00,0.034630,18.119043,0.000000e+00
B_AIR_RETAIL,-0.242885,0.077185,-3.146791,1.650732e-03,0.077770,-3.123132,1.789372e-03
B_AIR_TW,-0.492897,0.049136,-10.031203,0.000000e+00,0.049413,-9.975110,0.000000e+00
B_AIR_VALDEN,0.000150,0.000010,15.451960,0.000000e+00,0.000030,4.974583,6.538836e-07
